In [ ]:
!pip install wandb

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional: Download WordNet's extended multilingual data

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=15ccef5623c5e722b610ad09d783750b5493599ee37e3e6c91de672d63f07241
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
!pip install fuzzywuzzy

In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import wandb
from rouge_score import rouge_scorer
# from dotenv import load_dotenv
import os
from tabulate import tabulate
from nltk.translate.bleu_score import corpus_bleu
import sympy as sp
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.meteor_score import meteor_score
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score
from fuzzywuzzy import fuzz
# load_dotenv()

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
df = pd.read_csv('/content/cleaned_final_data - cleaned_final_data.csv')

# If 'Unnamed: 0' column still exists, you can drop it
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# 80% -> Training Data, 20% -> Testing Data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 90% -> Training Data, 10% -> Validation Data
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

In [ ]:
test_df.to_csv("test_df.csv")

In [ ]:
df.head()

,Question,Object,Answer
0,what is a yali ?,yali,a yali is a mythical creature found predominan...
1,what animals is a yali typically composed of ?,yali,a yali is typically depicted as a composite of...
2,what attributes does a yali symbolize ?,yali,"a yali symbolizes attributes like strength, pr..."
3,why are yalis considered unique ?,yali,yalis are unique because they do not adhere to...
4,which cultures have similar mythical creatures...,yali,cultures that have similar mythical creatures ...


In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

def calculate_max_length(column_name):
    df[column_name] = df[column_name].astype(str)
    return df[column_name].apply(lambda x: len(tokenizer.tokenize(x))).max()

max_length_question = calculate_max_length('Question')
max_length_object = calculate_max_length('Object')
max_length_answer = calculate_max_length('Answer')
# max_length_context = calculate_max_length('Context')

print(f"Maximum token length in the 'Question' column: {max_length_question + max_length_object}")
print(f"Maximum token length in the 'Answer' column: {max_length_answer}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Maximum token length in the 'Question' column: 39
Maximum token length in the 'Answer' column: 62


In [ ]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
class Dataset(Dataset):
    '''For Loading the datasetS! '''
    def __init__(self, data, question_max_length=50, answer_max_length=75):
        self.data = data
        self.tokenizer = tokenizer
        self.question_max_length = question_max_length
        self.answer_max_length = answer_max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question = self.data.iloc[idx, 0]
        object_name = self.data.iloc[idx, 1]
        answer = self.data.iloc[idx, 2]
        # context = self.data.iloc[idx, 3]
        combined = str(object_name) + ' ' + str(question)

        inputs = self.tokenizer(
            combined,
            max_length=self.question_max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        targets = self.tokenizer(
            answer,
            max_length=self.answer_max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        input_ids = inputs.input_ids.squeeze()
        attention_mask = inputs.attention_mask.squeeze()
        target_ids = targets.input_ids.squeeze()

        return {

            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': target_ids
        }

In [ ]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [ ]:
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [ ]:
train_dataset = Dataset(train_df)
val_dataset = Dataset(val_df)
test_dataset = Dataset(test_df)

In [ ]:
class VQA_Trainer:
    '''Class for Trainer Setup to Train the BART Model for VQA'''

    def __init__(self, model, train_dataloader, eval_dataloader, device, config):
        ''' Constructor '''
        self.model = model
        self.train_dataloader = train_dataloader
        self.eval_dataloader = eval_dataloader
        self.device = device
        self.tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
        self.scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)  # Initialize ROUGE scorer

        # Semantic similarity model
        self.semantic_model = SentenceTransformer('all-MiniLM-L6-v2')  # Pretrained sentence embedding model

        self.optimizer = AdamW(self.model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
        self.scheduler = get_linear_schedule_with_warmup(self.optimizer, num_warmup_steps=config['warmup_steps'], num_training_steps=len(self.train_dataloader) * config['epochs'])

        self.model.config.dropout = config['dropout']
        self.model.config.attention_dropout = config['attention_dropout']
        self.num_beams = config['num_beams']

        wandb.init(project=config['project_name'], config=config)
        wandb.watch(self.model, log="all")

    def evaluate(self):
        ''' For Evaluation at the End of Each Epoch '''
        self.model.eval()
        total_loss = 0
        predictions = []
        references = []
        token_level_accuracies = []
        exact_match_accuracies = []
        fuzzy_match_accuracies = []  # Add fuzzy match accuracy list
        semantic_similarities = []
        meteor_scores = []

        progress_bar = tqdm(self.eval_dataloader, desc="Evaluating")

        for batch in progress_bar:
            with torch.no_grad():
                inputs = {key: val.to(self.device) for key, val in batch.items()}
                outputs = self.model(**inputs)
                total_loss += outputs.loss.item()

                summary_ids = self.model.generate(inputs['input_ids'], max_length=80, num_beams=self.num_beams, early_stopping=True)
                decoded_preds = self.tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

                labels = batch['labels']
                labels = torch.where(labels != -100, labels, self.tokenizer.pad_token_id)
                decoded_refs = self.tokenizer.batch_decode(labels, skip_special_tokens=True)

                predictions.extend([pred.split() for pred in decoded_preds])
                references.extend([[ref.split()] for ref in decoded_refs])

                for pred, ref in zip(decoded_preds, decoded_refs):
                    pred_tokens = pred.split()
                    ref_tokens = ref.split()

                    # Token-level accuracy
                    token_accuracy = sum(1 for p, r in zip(pred_tokens, ref_tokens) if p == r) / max(len(ref_tokens), 1)
                    token_level_accuracies.append(token_accuracy)

                    # Exact Match Accuracy
                    exact_match_accuracy = 1 if pred.strip() == ref.strip() else 0
                    exact_match_accuracies.append(exact_match_accuracy)

                    # Fuzzy Match Accuracy (threshold set to 85 for near matches)
                    fuzzy_similarity = fuzz.ratio(pred.strip(), ref.strip())
                    fuzzy_match_accuracy = 1 if fuzzy_similarity >= 70 else 0
                    fuzzy_match_accuracies.append(fuzzy_match_accuracy)

                    # Semantic Similarity
                    pred_embedding = self.semantic_model.encode(pred)
                    ref_embedding = self.semantic_model.encode(ref)
                    similarity = cosine_similarity([pred_embedding], [ref_embedding])[0][0]
                    semantic_similarities.append(similarity)

                    # METEOR Score (use tokenized inputs)
                    meteor = meteor_score([ref_tokens], pred_tokens)  # Pass tokenized lists
                    meteor_scores.append(meteor)

        avg_loss = total_loss / len(self.eval_dataloader)
        perplexity = math.exp(avg_loss)

        bleu_score = corpus_bleu(references, predictions)

        rouge_1_f1_scores = []
        rouge_2_f1_scores = []
        rouge_l_f1_scores = []
        for pred, ref in zip(decoded_preds, decoded_refs):
            rouge_1 = self.scorer.score(ref, pred)['rouge1']
            rouge_2 = self.scorer.score(ref, pred)['rouge2']
            rouge_l = self.scorer.score(ref, pred)['rougeL']

            rouge_1_f1_scores.append(rouge_1.fmeasure)
            rouge_2_f1_scores.append(rouge_2.fmeasure)
            rouge_l_f1_scores.append(rouge_l.fmeasure)

        avg_token_level_accuracy = np.mean(token_level_accuracies)
        avg_exact_match_accuracy = np.mean(exact_match_accuracies)
        avg_fuzzy_match_accuracy = np.mean(fuzzy_match_accuracies)
        avg_semantic_similarity = np.mean(semantic_similarities)
        avg_meteor = np.mean(meteor_scores)

        avg_rouge_1_f1 = np.mean(rouge_1_f1_scores)
        avg_rouge_2_f1 = np.mean(rouge_2_f1_scores)
        avg_rouge_l_f1 = np.mean(rouge_l_f1_scores)

        df = pd.DataFrame({
            "Training Loss": [avg_loss],
            "ROUGE-1 (F1)": [avg_rouge_1_f1],
            "ROUGE-2 (F1)": [avg_rouge_2_f1],
            "ROUGE-L (F1)": [avg_rouge_l_f1],
            "Corpus BLEU": [bleu_score],
            "Semantic Similarity": [avg_semantic_similarity],
            "METEOR": [avg_meteor],
            "Token-Level Accuracy": [avg_token_level_accuracy],
            "Exact Match Accuracy": [avg_exact_match_accuracy],
            "Fuzzy Match Accuracy": [avg_fuzzy_match_accuracy],
            "Perplexity": [perplexity]
        })

        print(tabulate(df, headers="keys", tablefmt="psql"))

        metrics = {
            "Validation Loss": avg_loss,
            "perplexity": perplexity,
            "bleu": bleu_score,
            "rouge_1_f1": avg_rouge_1_f1,
            "rouge_2_f1": avg_rouge_2_f1,
            "rouge_l_f1": avg_rouge_l_f1,
            "semantic_similarity": avg_semantic_similarity,
            "meteor": avg_meteor,
            "token_level_accuracy": avg_token_level_accuracy,
            "exact_match_accuracy": avg_exact_match_accuracy,
            "fuzzy_match_accuracy": avg_fuzzy_match_accuracy,
        }

        wandb.log(metrics)

        return metrics

    def train_epoch(self):
        ''' To Train for Single Epoch '''
        self.model.train()
        for batch in tqdm(self.train_dataloader, desc="Training"):
            self.optimizer.zero_grad()
            inputs = {key: val.to(self.device) for key, val in batch.items()}
            outputs = self.model(**inputs)
            loss = outputs.loss
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()

            wandb.log({"train_loss": loss.item()})

    def train(self, epochs):
        ''' To Train for N Number of Epochs Passed from User '''
        for epoch in range(epochs):
            print(f'Epoch {epoch+1}/{epochs}')
            self.train_epoch()
            metrics = self.evaluate()
            print(f"Metrics: {metrics}")
            torch.save(self.model.state_dict(), f"model_{epoch}.pth")


In [ ]:
api_key = os.getenv('API_KEY')
!wandb login --relogin $api_key

Traceback (most recent call last):
  File "/usr/local/bin/wandb", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/cli/cli.py", line 104, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/cli/cli.py", line 254, in login
    wandb.login(
  File "/usr/local/lib/python3.1

In [ ]:
config = {
     "epochs" : 20,
     "model_name": "facebook/bart-base",
     "project_name": "VQA_BART_final_finetuning",
     'learning_rate': 0.00021221658403100733,
     'weight_decay': 0.00001005975393896157,
     'dropout': 0.1991270941070338,
     'attention_dropout': 0.3192191116986305,
     'num_beams': 8,
     'batch_size': 8,
     'warmup_steps': 784
}

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BartForConditionalGeneration.from_pretrained(config['model_name']).to(device)

In [ ]:
trainer = VQA_Trainer(model, train_dataloader, val_dataloader, device=device, config=config)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
trainer.train(config['epochs'])

Epoch 1/20


Evaluating: 100%|██████████| 90/90 [01:43<00:00,  1.15s/it]


+----+-----------------+----------------+----------------+----------------+---------------+-----------------------+----------+------------------------+------------------------+------------------------+--------------+
|    |   Training Loss |   ROUGE-1 (F1) |   ROUGE-2 (F1) |   ROUGE-L (F1) |   Corpus BLEU |   Semantic Similarity |   METEOR |   Token-Level Accuracy |   Exact Match Accuracy |   Fuzzy Match Accuracy |   Perplexity |
|----+-----------------+----------------+----------------+----------------+---------------+-----------------------+----------+------------------------+------------------------+------------------------+--------------|
|  0 |        0.607843 |        0.29391 |       0.109795 |       0.267237 |     0.0998256 |              0.644935 | 0.267388 |               0.121046 |             0.00139082 |              0.0667594 |      1.83647 |
+----+-----------------+----------------+----------------+----------------+---------------+-----------------------+----------+------

Evaluating: 100%|██████████| 90/90 [01:52<00:00,  1.25s/it]


+----+-----------------+----------------+----------------+----------------+---------------+-----------------------+----------+------------------------+------------------------+------------------------+--------------+
|    |   Training Loss |   ROUGE-1 (F1) |   ROUGE-2 (F1) |   ROUGE-L (F1) |   Corpus BLEU |   Semantic Similarity |   METEOR |   Token-Level Accuracy |   Exact Match Accuracy |   Fuzzy Match Accuracy |   Perplexity |
|----+-----------------+----------------+----------------+----------------+---------------+-----------------------+----------+------------------------+------------------------+------------------------+--------------|
|  0 |        0.456614 |       0.416905 |        0.18193 |       0.359442 |      0.187211 |              0.734724 |  0.39349 |               0.194496 |             0.00834492 |               0.215577 |      1.57872 |
+----+-----------------+----------------+----------------+----------------+---------------+-----------------------+----------+------

...

+----+-----------------+----------------+----------------+----------------+---------------+-----------------------+----------+------------------------+------------------------+------------------------+--------------+
|    |   Training Loss |   ROUGE-1 (F1) |   ROUGE-2 (F1) |   ROUGE-L (F1) |   Corpus BLEU |   Semantic Similarity |   METEOR |   Token-Level Accuracy |   Exact Match Accuracy |   Fuzzy Match Accuracy |   Perplexity |
|----+-----------------+----------------+----------------+----------------+---------------+-----------------------+----------+------------------------+------------------------+------------------------+--------------|
|  0 |        0.341105 |       0.889363 |       0.769208 |       0.889363 |      0.590028 |              0.879037 | 0.764649 |               0.666921 |               0.038943 |               0.792768 |       1.4065 |
+----+-----------------+----------------+----------------+----------------+---------------+-----------------------+----------+------

Evaluating: 100%|██████████| 90/90 [01:54<00:00,  1.27s/it]


+----+-----------------+----------------+----------------+----------------+---------------+-----------------------+----------+------------------------+------------------------+------------------------+--------------+
|    |   Training Loss |   ROUGE-1 (F1) |   ROUGE-2 (F1) |   ROUGE-L (F1) |   Corpus BLEU |   Semantic Similarity |   METEOR |   Token-Level Accuracy |   Exact Match Accuracy |   Fuzzy Match Accuracy |   Perplexity |
|----+-----------------+----------------+----------------+----------------+---------------+-----------------------+----------+------------------------+------------------------+------------------------+--------------|
|  0 |        0.345951 |       0.905129 |       0.797198 |       0.905129 |        0.6099 |              0.886832 | 0.783722 |               0.693947 |              0.0417246 |                0.81363 |      1.41333 |
+----+-----------------+----------------+----------------+----------------+---------------+-----------------------+----------+------

Training:  44%|████▎     | 353/809 [01:14<01:36,  4.74it/s]


KeyboardInterrupt: 

In [ ]:
wandb.finish()

Validation Loss,█▄▂▁▁▁▁▁▁▁▁▁▁▁
bleu,▁▂▃▄▄▅▆▆▆▇▇▇██
exact_match_accuracy,▁▂▂▂▃▆▄▇▇▇▆▇██
fuzzy_match_accuracy,▁▂▃▄▅▅▆▇▇▇▇███
meteor,▁▃▄▄▅▅▆▇▇▇████
perplexity,█▄▂▁▁▁▁▁▁▁▁▁▁▁
rouge_1_f1,▁▂▂▄▅▅▅▅▇▇▇███
rouge_2_f1,▁▂▂▃▄▅▄▅▇▇▆███
rouge_l_f1,▁▂▂▄▅▅▅▅▇▇▇███
semantic_similarity,▁▄▅▅▆▆▇▇▇▇████
token_level_accuracy,▁▂▃▃▄▅▆▆▆▇▇▇██


In [ ]:
# Inference code

# Load the tokenizer and model (make sure these paths match your setup)
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
checkpoint_path = '/content/model_13.pth'

# Load your trained model checkpoint
def load_checkpoint(model, file_path):
    checkpoint = torch.load(file_path, map_location=device)
    print('checkpoint keys: ', checkpoint.keys())
    model.load_state_dict(checkpoint)
    print(f"Checkpoint loaded from {file_path}")

load_checkpoint(model, checkpoint_path)
model.eval()

<ipython-input-23-5f4a78a0c64b>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(file_path, map_location=device)


checkpoint keys:  odict_keys(['final_logits_bias', 'model.shared.weight', 'model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.encoder.layers.0.self_attn.k_proj.weight', 'model.encoder.layers.0.self_attn.k_proj.bias', 'model.encoder.layers.0.self_attn.v_proj.weight', 'model.encoder.layers.0.self_attn.v_proj.bias', 'model.encoder.layers.0.self_attn.q_proj.weight', 'model.encoder.layers.0.self_attn.q_proj.bias', 'model.encoder.layers.0.self_attn.out_proj.weight', 'model.encoder.layers.0.self_attn.out_proj.bias', 'model.encoder.layers.0.self_attn_layer_norm.weight', 'model.encoder.layers.0.self_attn_layer_norm.bias', 'model.encoder.layers.0.fc1.weight', 'model.encoder.layers.0.fc1.bias', 'model.encoder.layers.0.fc2.weight', 'model.encoder.layers.0.fc2.bias', 'model.encoder.layers.0.final_layer_norm.weight', 'model.encoder.layers.0.final_layer_norm.bias', 'model.encoder.layers.1.self_attn.k_proj.weight', 'model.encoder.layers.1.self_attn.k_proj.bias', 'model.

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [ ]:
# Function to generate answers based on input questions
def ask_question(question, object_name, max_length=80):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    combined_input = f"{object_name} {question}"

    inputs = tokenizer(
        combined_input,
        return_tensors="pt",
        max_length=50,
        truncation=True
    ).to(device)

    answer_ids = model.generate(
        inputs['input_ids'],
        max_length=max_length,
        num_beams=8,
        early_stopping=True
    )
    answer = tokenizer.decode(answer_ids[0], skip_special_tokens=True)
    return answer

In [ ]:
test_df = pd.read_csv('/content/test_df.csv')
test_df.head()

,Unnamed: 0,Question,Object,Answer
0,7974,in what way does bhairava represent the ruiner...,kala bhairav,his trench skin tone and formidable characteri...
1,169,what is the historical origin of the kumari-pu...,taleju temple,it dates back to the 17th century through virg...
2,866,how do hindu texts vary in their depiction of ...,garuda,hindu texts vary in their details of garuda's ...
3,2614,what was the ambition of the middle child in t...,boudhanath,he hoped to arise from a lotus and defend the ...
4,733,who intervened when hanuman tried to eat the s...,hanuman idol,"the king of gods, indra, intervened and struck..."


In [ ]:
import pandas as pd

# Retrieve all the necessary columns from the DataFrame
sampled_questions = test_df['Question']
sampled_objects = test_df['Object']
sampled_original_answers = test_df['Answer']  # Original answers

answers = []
for question, object_name, original_answer in zip(sampled_questions, sampled_objects, sampled_original_answers):
    generated_answer = ask_question(question, object_name)
    answers.append({
        "Question": question,
        "Original Answer": original_answer,
        "Generated Answer": generated_answer
    })

# Convert the answers list into a DataFrame
answers_df = pd.DataFrame(answers)

# Optionally, you can save this DataFrame to a CSV file
answers_df.to_csv('generated_answers_final.csv', index=False)

# Print the DataFrame to verify
print(answers_df)

                                               Question  \
0     in what way does bhairava represent the ruiner...   
1     what is the historical origin of the kumari-pu...   
2     how do hindu texts vary in their depiction of ...   
3     what was the ambition of the middle child in t...   
4     who intervened when hanuman tried to eat the s...   
...                                                 ...   
1793               when was the taleju bell installed ?   
1794  what direction do pilgrims walk when turning p...   
1795             what qualities does a yali represent ?   
1796  how are the grids utilized in aankhi jhyal win...   
1797          what do bhairava's attributes symbolize ?   

                                        Original Answer  \
0     his trench skin tone and formidable characteri...   
1     it dates back to the 17th century through virg...   
2     hindu texts vary in their details of garuda's ...   
3     he hoped to arise from a lotus and defend the ...

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
